In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [0]:
from pyspark.ml.recommendation import ALS

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [8]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [10]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [0]:
training, test = data.randomSplit([0.8,0.2])

In [0]:
als = ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [0]:
model = als.fit(training)

In [14]:
predictions = model.transform(test)
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|    13| 1.5194738|
|     31|   3.0|     7| 1.9451195|
|     31|   2.0|    25| 3.8665628|
|     31|   3.0|    14| 2.8040915|
|     85|   3.0|     1| 0.8154763|
|     85|   3.0|     6|  2.313168|
|     85|   2.0|    20| -0.376954|
|     85|   5.0|     8| 1.3570306|
|     85|   1.0|     2| 3.7001705|
|     65|   2.0|     5| 1.7778337|
|     65|   1.0|     4|-0.7954771|
|     53|   1.0|     6| 2.9969614|
|     53|   1.0|     7| 1.4965177|
|     78|   1.0|    17| 1.0335387|
|     78|   1.0|    24| 0.4653482|
|     78|   1.0|    11|  0.726068|
|     34|   1.0|    19| 2.5724607|
|     34|   1.0|    15| 2.6836839|
|     34|   4.0|     2| 1.8256531|
|     81|   5.0|    28| 0.6097967|
+-------+------+------+----------+
only showing top 20 rows



In [0]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [0]:
rmse = evaluator.evaluate(predictions)

In [17]:
print(rmse)

1.4869123065744994


### Recommend Single User

In [0]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [27]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     22|    11|
|     25|    11|
|     36|    11|
|     47|    11|
|     50|    11|
|     59|    11|
|     67|    11|
|     72|    11|
|     78|    11|
|     80|    11|
|     99|    11|
+-------+------+



In [0]:
recommendations = model.transform(single_user)

In [29]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     36|    11|  3.5271435|
|     22|    11|  3.3225474|
|     80|    11|  3.0185175|
|     50|    11|  2.1668868|
|     47|    11|  1.1818218|
|     99|    11|  1.1172342|
|     67|    11|   0.793117|
|     78|    11|   0.726068|
|     25|    11|  0.6304986|
|     59|    11|  0.5876398|
|     72|    11|-0.23257433|
+-------+------+-----------+

